<a href="https://colab.research.google.com/github/Josephjyinn516/Imaging-Processing/blob/main/ISEAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import numpy as np

# Load the image using Pillow
image_path = 'download (5).jpeg'
original_image = Image.open(image_path)

# Enhance color to make it more vibrant
enhancer = ImageEnhance.Color(original_image)
enhanced_image = enhancer.enhance(1.5)  # Adjust the factor to control the enhancement

# Apply a vignette effect
width, height = original_image.size
mask = Image.new('L', (width, height), 0)
x, y = width / 2, height / 2
for i in range(width):
    for j in range(height):
        mask.putpixel((i, j), int(255 * (1 - ((i - x)**2 + (j - y)**2) / (1.5 * width * height))))
vignette_image = Image.composite(enhanced_image, original_image, mask)

# Convert the image to an array for OpenCV processing
cv_image = np.array(vignette_image)

# Sharpen the image
kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
sharpened_image = cv2.filter2D(cv_image, -1, kernel)

# Apply a slight blur effect on the edges of the feathers to give a sense of motion
# Assuming the feathers are located at the bottom half of the image
feather_region = sharpened_image[height//2:, :]
blurred_feathers = cv2.GaussianBlur(feather_region, (7, 7), 0)
sharpened_image[height//2:, :] = blurred_feathers

# Save the final image
final_image = Image.fromarray(sharpened_image)
final_image.save('enhanced_image.jpg')


In [4]:
from PIL import Image, ImageEnhance
import imageio

# Load the image
image_path = 'castle(2).jpg'
original_image = Image.open(image_path)

# Adjust brightness
brightness = ImageEnhance.Brightness(original_image)
darker_image = brightness.enhance(0.5)  # Make the image darker

# Adjust contrast
contrast = ImageEnhance.Contrast(darker_image)
more_contrast_image = contrast.enhance(1.5)  # Increase contrast

# Adjust saturation
saturation = ImageEnhance.Color(more_contrast_image)
gloomy_image = saturation.enhance(0.7)  # Reduce color saturation

# Load the overlay image
overlay_image_path = 'traveller-removebg-preview.png'
overlay_image = Image.open(overlay_image_path).convert("RGBA")

# Set the transparency level for the overlay
transparency = 0.8  # Adjust as needed

# Manually set the desired overlay size (e.g., 200x200 pixels)
desired_overlay_width = 179
desired_overlay_height = 330

# Resize the overlay image
overlay_image = overlay_image.resize((desired_overlay_width, desired_overlay_height), Image.ANTIALIAS)

# Create a mask using the alpha channel of the overlay image
mask = overlay_image.split()[3].point(lambda p: p * transparency)

# Calculate the position for the bottom-left corner
overlay_x = 0
overlay_y = gloomy_image.height - overlay_image.height

# Paste the overlay onto the original image with the mask
gloomy_image.paste(overlay_image, (overlay_x, overlay_y), mask)

# Save the result
gloomy_image.save('final_image_with_overlay.jpg')
gloomy_image_path = 'final_image_with_overlay.jpg'
gloomy_image = Image.open(gloomy_image_path)

fog_gif_path = 'fog.gif'
fog_gif = imageio.mimread(fog_gif_path)

combined_frames = []

for frame in fog_gif:
    fog_frame = Image.fromarray(frame).resize(gloomy_image.size,
Image.ANTIALIAS)

        # Blend the fog frame with the gloomy image
    combined_frame = Image.blend(gloomy_image, fog_frame, alpha=0.3)  # Adjust alpha as needed

    # Append the blended frame to the list
    combined_frames.append(combined_frame)

# Save the frames as a new GIF
combined_frames[0].save('gloomy_with_fog.gif', save_all=True,
append_images=combined_frames[1:], loop=0, duration=100)


# Display the result
gloomy_image.show()


<ipython-input-4-436cad157e88>:32: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  overlay_image = overlay_image.resize((desired_overlay_width, desired_overlay_height), Image.ANTIALIAS)
<ipython-input-4-436cad157e88>:56: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  Image.ANTIALIAS)


In [26]:
from PIL import Image, ImageSequence, ImageOps

def change_wing_color(gif_path, overlay_image_path, output_path):
    # Open the original GIF
    with Image.open(gif_path) as img:
        overlay = Image.open(overlay_image_path)
        # Ensure overlay is in RGBA mode for transparency
        overlay = overlay.convert('RGBA')
        # Create a list to hold processed frames
        frames = []
        # Iterate over each frame in the animated GIF
        for frame in ImageSequence.Iterator(gif):
            # Convert the frame to RGB
            rgb_frame = frame.convert('RGB')
            # Invert colors to change white to black and vice versa
            inverted_frame = ImageOps.invert(rgb_frame)
            # Append the processed frame to the list
            frames.append(inverted_frame)

        # Save frames as a new GIF
        frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0)

# Path to your original GIF
gif_path = 'wings.gif'
# Path to your overlay image (in JPG format)
overlay_image_path = 'sephiroth.jpg'
# Path to your output GIF
output_path = 'new_output_with_overlay.gif'



In [3]:
from PIL import Image, ImageEnhance, ImageDraw, ImageFilter, ImageSequence
import random
import numpy as np

# Function to change the color of the GIF
def change_gif_color(gif_path, target_color):
    gif = Image.open(gif_path)
    frames = []

    for frame in ImageSequence.Iterator(gif):
        frame = frame.convert('RGBA')
        data = np.array(frame)

        # Replace white (255, 255, 255, 255) with the target color (0, 0, 0, 255)
        r, g, b, a = data.T
        white_areas = (r == 255) & (g == 255) & (b == 255) & (a == 255)
        data[..., :-1][white_areas.T] = target_color[:-1]
        data[..., -1][white_areas.T] = target_color[-1]

        new_frame = Image.fromarray(data)
        frames.append(new_frame)

    # Save the modified GIF
    frames[0].save('black_wings.gif', save_all=True, append_images=frames[1:], loop=0, duration=gif.info['duration'], dispose=2)
    return 'black_wings.gif'

# Function to enhance and resize the image
def enhance_and_resize_image(image_path, target_size):
    image = Image.open(image_path)

    # Resize the image
    resized_image = image.resize(target_size, Image.LANCZOS)

    # Enhance brightness
    enhancer = ImageEnhance.Brightness(resized_image)
    enhanced_image = enhancer.enhance(1.2)

    # Enhance contrast
    enhancer = ImageEnhance.Contrast(enhanced_image)
    enhanced_image = enhancer.enhance(1.3)

    # Enhance sharpness
    enhancer = ImageEnhance.Sharpness(enhanced_image)
    enhanced_image = enhancer.enhance(2.0)

    enhanced_image.save('resized_enhanced_sephiroth.jpg')
    return 'resized_enhanced_sephiroth.jpg'

new_width = 498
new_height = 280
# Specify the target size for the resized image
target_size = (new_width, new_height)  # Set the new width and height'


# Function to overlay the enhanced image on the processed GIF
def overlay_image_on_gif(image_path, gif_path, output_path, resize_factor=1, position=(0, 0)):
    enhanced_image = Image.open(image_path).convert('RGBA')

    # Resize enhanced image
    new_size = (int(enhanced_image.width * resize_factor), int(enhanced_image.height * resize_factor))
    enhanced_image = enhanced_image.resize(new_size, Image.LANCZOS)

    gif = Image.open(gif_path)
    frames = []

    for frame in ImageSequence.Iterator(gif):
        frame = frame.convert('RGBA')

        # Create a new blank image with the same size as the frame
        new_frame = Image.new('RGBA', frame.size)

        # Paste the frame and the enhanced image onto the new image
        new_frame.paste(frame, (0, 0))
        new_frame.paste(enhanced_image, position, enhanced_image)

        frames.append(new_frame)

    # Save the new GIF
    frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0, duration=gif.info['duration'], dispose=2)

# Step 1: Change the color of the wings in the GIF
black_wings_gif_path = change_gif_color('wings.gif', (0, 0, 0, 255))

# Step 2: Enhance the Sephiroth image
enhanced_resized_sephiroth_image_path = enhance_and_resize_image('sephiroth.jpg', target_size)

# Step 3: Overlay the enhanced Sephiroth image on the processed wings GIF
overlay_image_on_gif(enhanced_resized_sephiroth_image_path, black_wings_gif_path, 'final_output.gif', resize_factor=0.5, position=(50, 50))


In [6]:
from PIL import Image, ImageEnhance, ImageDraw, ImageFilter, ImageSequence
import random
import PIL

# Increase the maximum allowed pixels to avoid DecompressionBombError
PIL.Image.MAX_IMAGE_PIXELS = None

# Load the image
image_path = 'treasure chest.jpg'
original_image = Image.open(image_path)

# Enhance the image to make it shiny
enhancer = ImageEnhance.Brightness(original_image)
brighter_image = enhancer.enhance(1.5)  # Increase brightness to make it shiny

# Add a glow effect
glow = brighter_image.filter(ImageFilter.GaussianBlur(radius=2))
combined = Image.blend(brighter_image, glow, alpha=0.2)  # Adjust alpha for glow intensity

# Increase contrast to make the colors pop
contrast = ImageEnhance.Contrast(combined)
contrasted_image = contrast.enhance(1.5)  # Adjust contrast

# Draw sparkles
draw = ImageDraw.Draw(contrasted_image)
for _ in range(50):  # Number of sparkles
    x = random.randint(0, contrasted_image.width)
    y = random.randint(0, contrasted_image.height)
    size = random.randint(1, 3)
    draw.line((x - size, y - size, x + size, y + size), fill='white', width=1)
    draw.line((x - size, y + size, x + size, y - size), fill='white', width=1)

# Add transparency to the enhanced image
alpha = 128  # Adjust transparency level (0 = transparent, 255 = opaque)
contrasted_image.putalpha(alpha)

# Save the modified image
enhanced_image_path = 'path_to_save_shiny_treasure_image.png'
contrasted_image.save(enhanced_image_path)

# Function to overlay a GIF on the enhanced image
def overlay_gif_on_image(enhanced_image_path, gif_path, output_path):
    try:
        # Load the enhanced image
        enhanced_image = Image.open(enhanced_image_path).convert('RGBA')
        enhanced_size = enhanced_image.size
        print(f"Enhanced image loaded successfully: {enhanced_image_path}")
    except Exception as e:
        print(f"Error loading enhanced image: {e}")
        return

    try:
        # Load the GIF
        gif = Image.open(gif_path)
        print(f"GIF loaded successfully: {gif_path}")
    except Exception as e:
        print(f"Error loading GIF: {e}")
        return

    # Create a list to hold the new frames
    new_frames = []

    # Iterate over each frame in the GIF
    for i, frame in enumerate(ImageSequence.Iterator(gif)):
        try:
            # Resize the frame to match the size of the enhanced image
            frame = frame.resize(enhanced_size, Image.LANCZOS).convert('RGBA')
            print(f"Frame {i} resized successfully")

            # Overlay the enhanced image onto the frame
            combined_frame = Image.alpha_composite(frame, enhanced_image)
            print(f"Frame {i} overlaid successfully")

            # Append the combined frame to the list
            new_frames.append(combined_frame)
        except Exception as e:
            print(f"Error processing frame {i}: {e}")
            continue

    # Check if we have at least one frame to save
    if new_frames:
        # Save the new frames as a GIF
        try:
            new_frames[0].save(output_path, save_all=True, append_images=new_frames[1:], loop=0, duration=gif.info.get('duration', 100), dispose=2)
            print(f"New GIF saved successfully: {output_path}")
        except Exception as e:
            print(f"Error saving new GIF: {e}")
    else:
        print("No frames were processed successfully, GIF not created.")

# Call the modified function with the appropriate file paths
overlay_gif_on_image(enhanced_image_path, 'shiny.gif', 'enhanced_overlayed_gif.gif')


Enhanced image loaded successfully: path_to_save_shiny_treasure_image.png
GIF loaded successfully: shiny.gif
Frame 0 resized successfully
Frame 0 overlaid successfully
Frame 1 resized successfully
Frame 1 overlaid successfully
Frame 2 resized successfully
Frame 2 overlaid successfully
New GIF saved successfully: enhanced_overlayed_gif.gif


In [ ]:
from PIL import Image, ImageEnhance, ImageFilter

# Load the image
image_path = 'sephorith_death.jpeg'
original_image = Image.open(image_path)

# Desaturate the colors to give a darker, more defeated look
converter = ImageEnhance.Color(original_image)
desaturated_image = converter.enhance(0.5)  # Adjust the factor to desaturate the colors

# Increase shadows to emphasize the sense of loss
enhancer = ImageEnhance.Brightness(desaturated_image)
darker_image = enhancer.enhance(0.6)  # Adjust the factor to darken the image

# Apply a filter to create a blur effect, simulating damage
blurred_image = darker_image.filter(ImageFilter.GaussianBlur(radius=2))

# Save the modified image
blurred_image.save('path_to_save_modified_image.jpg')


In [ ]:
from PIL import Image, ImageEnhance, ImageFilter, ImageDraw
import cv2
import numpy as np

# Load the image using Pillow
image_path = 'final_battle.jpeg'
original_image = Image.open(image_path)

# Enhance textures on armor and weapons
enhancer = ImageEnhance.Sharpness(original_image)
sharpened_image = enhancer.enhance(2.0)  # Adjust the factor to enhance sharpness

# Increase contrast for a dramatic effect
contrast = ImageEnhance.Contrast(sharpened_image)
contrasted_image = contrast.enhance(1.5)  # Adjust the factor to increase contrast

# Apply dynamic lighting with a radial gradient
width, height = original_image.size
lighting_layer = Image.new('RGB', (width, height))
draw = ImageDraw.Draw(lighting_layer)
for i in range(width):
    for j in range(height):
        distance = np.sqrt((width / 2 - i)**2 + (height / 2 - j)**2)
        draw.point((i, j), fill=(255, 255, 255, max(255 - int(distance / 2), 0)))
lighting_image = Image.blend(contrasted_image, lighting_layer, alpha=0.1)  # Adjust alpha for lighting intensity

# Convert the image to an array for OpenCV processing
cv_image = np.array(lighting_image)

# Add motion blur to give a sense of movement
size = 15  # Size of the kernel
kernel_motion_blur = np.zeros((size, size))
kernel_motion_blur[int((size - 1) / 2), :] = np.ones(size)
kernel_motion_blur = kernel_motion_blur / size
motion_blur_image = cv2.filter2D(cv_image, -1, kernel_motion_blur)

# Add fog or mist for atmospheric effect
foggy_image = cv2.addWeighted(motion_blur_image, 0.7, np.full(motion_blur_image.shape, 255, dtype=motion_blur_image.dtype), 0.3, 0)

# Add sparks or small debris to illustrate the intensity of combat
for _ in range(100):  # Number of sparks
    x, y = np.random.randint(0, width), np.random.randint(0, height)
    foggy_image[y:y+2, x:x+2] = [0, 255, 255]  # Spark color in BGR format

# Save the final image
final_image = Image.fromarray(foggy_image)
final_image.save('path_to_save_enhanced_battle_image.jpg')


In [ ]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('victory.jpeg')

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to create a "glow" effect
blurred = cv2.GaussianBlur(gray, (21, 21), 0)

# Enhance the glow effect by increasing the intensity
intense_glow = cv2.addWeighted(gray, 0.5, blurred, 0.5, 0)

# Coordinates of the sword's position
x1, y1 = 100, 200  # Replace 100, 200 with the actual x, y coordinates of the top-left corner of the sword
x2, y2 = 200, 300  # Replace 200, 300 with the actual x, y coordinates of the bottom-right corner of the sword

# Create a mask for the sword to apply the glow effect
# This is a placeholder; you'll need to create a mask based on your image's sword position
mask = np.zeros_like(gray)
cv2.rectangle(mask, (x1, y1), (x2, y2), (255, 255, 255), -1)  # Replace with sword coordinates

# Apply the glow effect to the sword area
sword_glow = cv2.bitwise_and(intense_glow, intense_glow, mask=mask)

# Convert back to BGR to add colored effects
sword_glow_colored = cv2.cvtColor(sword_glow, cv2.COLOR_GRAY2BGR)

# Add the glow effect back to the original image
final_image = cv2.addWeighted(image, 1, sword_glow_colored, 0.3, 0)

# Save the enhanced image
cv2.imwrite('path_to_save_enhanced_image.jpg', final_image)


True

In [ ]:
import imageio.v2 as imageio
import os

# List of image filenames (replace with your actual image filenames)
filenames = ["frame_00_delay-0.08s.jpg", "frame_03_delay-0.08s.jpg", "frame_04_delay-0.08s.jpg", "frame_06_delay-0.08s.jpg", "frame_08_delay-0.08s.jpg", "frame_10_delay-0.08s.jpg", "frame_11_delay-0.08s.jpg", "frame_13_delay-0.08s.jpg", "frame_15_delay-0.08s.jpg", "frame_17_delay-0.08s.jpg", "frame_18_delay-0.08s.jpg", "frame_20_delay-0.08s.jpg", "frame_22_delay-0.08s.jpg", "frame_24_delay-0.08s.jpg", "frame_26_delay-0.08s.jpg", "frame_28_delay-0.08s.jpg", "frame_30_delay-0.08s.jpg", "frame_32_delay-0.08s.jpg", "frame_33_delay-0.08s.jpg", "frame_35_delay-0.08s.jpg", "frame_37_delay-0.08s.jpg", "frame_39_delay-0.08s.jpg", "frame_41_delay-0.08s.jpg", "frame_43_delay-0.08s.jpg", "frame_45_delay-0.08s.jpg", "frame_47_delay-0.08s.jpg", "frame_49_delay-0.08s.jpg", "frame_51_delay-0.08s.jpg", "frame_53_delay-0.08s.jpg", "frame_55_delay-0.08s.jpg", "frame_57_delay-0.08s.jpg", "frame_59_delay-0.08s.jpg", "frame_61_delay-0.08s.jpg", "frame_63_delay-0.08s.jpg", "frame_65_delay-0.08s.jpg", "frame_67_delay-0.08s.jpg", "frame_69_delay-0.08s.jpg", "frame_71_delay-0.08s.jpg", "frame_72_delay-0.08s.jpg", "frame_74_delay-0.08s.jpg", "frame_76_delay-0.08s.jpg"]

# Read each image and create a list of frames
images = [imageio.imread(filename) for filename in filenames]

# Specify the full path to the directory where you want to save the GIF
output_directory = "/Users/User/Downloads"  # Replace with your actual path

# Check if the directory exists, if not, create it
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Save the frames as an animated GIF
output_path = os.path.join(output_directory, "animation.gif")
imageio.mimsave(output_path, images, duration=2.0)

print(f"Animated GIF saved at: {output_path}")

# Save the frames as an animated GIF
# imageio.mimsave("/path/to/animation.gif", images, duration=2.0)

# import glob
# from PIL import Image

# def make_gif(frame_folder):
    # Load all JPG images in the specified folder
    # frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*.JPG")]

    # Get the first frame
    # frame_one = frames[0]

    # Save frames as an animated GIF
    # frame_one.save("my_awesome.gif", format="GIF", append_images=frames, save_all=True, duration=100, loop=0)

# if __name__ == "__main__":
    # make_gif("/path/to/images")



Animated GIF saved at: /Users/User/Downloads/animation.gif


In [ ]:
from PIL import Image, ImageSequence

# Load your image
image_path = 'fighting.jpg'
image = Image.open(image_path)

# Create a list to hold the frames
frames = []

# Let's assume we want to create a GIF with 10 frames
# We will apply a transformation to create movement
for i in range(10):
    # Apply some transformation to the image (e.g., rotate or resize)
    new_frame = image.rotate(i * 36)  # Rotating the image by 36 degrees each time
    frames.append(new_frame)

# Save the frames as an animated GIF
gif_path = 'path_to_save_your_gif.gif'
frames[0].save(gif_path, format='GIF', append_images=frames[1:], save_all=True, duration=100, loop=0)
